# A notebook for computing evaporation rates from PIPS data and radar retrievals

In [ ]:
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
import pyPIPS.timemodule as ptime
# from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import cartopy.crs as ccrs
import cartopy.feature as feature
# from natsort import natsorted
from pprint import pprint
import warnings
warnings.simplefilter('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# date = '052516' # '053122' # '030622' # '061416'

PIPS_base_dir = "/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/"
deployment_name = "IOP2_033022"
PIPS_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')

PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
parsivel_interval = 10
intervalstr = f'{parsivel_interval:d}S'
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]

conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_dir, conv_filename) for conv_filename in conv_filenames]

parsivel_ds_read_dict = {}
conv_ds_read_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    parsivel_ds_read_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    conv_ds_read_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
for PIPS_name in PIPS_names:
    parsivel_ds = parsivel_ds_read_dict[PIPS_name]
    print(parsivel_ds['time'][0], parsivel_ds['time'][-1])

In [ ]:
# Restrict to certain time range
# start_time = '2022-05-31T23:00' # '2022-03-07T00:00'
# end_time = '2022-06-01T00:05' # '2022-03-08T00:00'
# start_time = '2022-03-30T23:40'
# end_time = '2022-03-31T01:30'
# start_time = '2023-03-12T00:15'
# end_time = '2023-03-12T14:00'
# start_time = '2023-02-22T16:00'
# end_time = '2023-02-23T01:00'
start_time = '2023-03-16T17:45'
end_time = '2023-03-17T15:05'

if False:
    parsivel_ds_dict = {}
    conv_ds_dict = {}
    for PIPS_name in PIPS_names:
        parsivel_ds_dict[PIPS_name] = parsivel_ds_read_dict[PIPS_name].sel(time=slice(start_time, end_time))
        conv_ds_dict[PIPS_name] = conv_ds_read_dict[PIPS_name].sel(time=slice(start_time, end_time))
else:
    parsivel_ds_dict = parsivel_ds_read_dict
    conv_ds_dict = conv_ds_read_dict

In [ ]:
parsivel_ds_dict['PIPS1A']

In [ ]:
PIPS_to_plot = 'PIPS1A'
parsivel_ds = parsivel_ds_dict[PIPS_to_plot]

In [ ]:
N0_TMM234 = parsivel_ds['DSD_TMM234_roqc'].sel(parameter='N0')
lamda_TMM234 = parsivel_ds['DSD_TMM234_roqc'].sel(parameter='lamda')
alpha_TMM234 = parsivel_ds['DSD_TMM234_roqc'].sel(parameter='alpha')

N0_MM234 = parsivel_ds['DSD_MM234_roqc'].sel(parameter='N0')
lamda_MM234 = parsivel_ds['DSD_MM234_roqc'].sel(parameter='lamda')
alpha_MM234 = parsivel_ds['DSD_MM234_roqc'].sel(parameter='alpha')

N0_retr = parsivel_ds['N0_retr_Z01_4dB_roqc']
lamda_retr = parsivel_ds['lamda_retr_Z01_4dB_roqc']
alpha_retr = parsivel_ds['mu_retr_Z01_4dB_roqc']

N0_retr = N0_retr * 1000.**(1. + alpha_retr)
lamda_retr = lamda_retr * 1000.

rho = parsivel_ds['rho']
T = parsivel_ds['fasttemp'] + 273.15
RH = parsivel_ds['RH_derived']
p = parsivel_ds['pressure'] * 100.


In [ ]:
QREVP, EVP_cool_rate = dsd.calc_evap(rho, T, p, RH, N0_TMM234, lamda_TMM234, alpha_TMM234)
QREVP2, EVP_cool_rate2 = dsd.calc_evap(rho, T, p, RH, N0_MM234, lamda_MM234, alpha_MM234)
QREVP3, EVP_cool_rate3 = dsd.calc_evap(rho, T, p, RH, N0_retr, lamda_retr, alpha_retr)

In [ ]:
EVP_cool_rate.plot()
EVP_cool_rate2.plot()
EVP_cool_rate3.plot()